In [1]:
import open3d as o3d
import numpy as np
from numpy.random import default_rng
import os
import sys
import math
import random
from time import sleep
# monkey patches visualization and provides helpers to load geometries
#sys.path.append('..')
#import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
#o3dtut.interactive = not "CI" in os.environ

In [2]:
class cube:
    def __init__(self, x, y, z, w, l, h, thetaX, thetaY, thetaZ):
        self.x = x
        self.y = y
        self.z = z
        self.w = w
        self.l = l
        self.h = h
        self.thetaX = thetaX
        self.thetaY = thetaY
        self.thetaZ = thetaZ
        
        thresh = 0.15
         
        self.centerx = x + w/2
        self.centery = y + l/2
        self.centerz = z + h/2
        self.pts_total = 0
        self.inliers_total =0
        ptA = [x, y, z]
        ptB = [x+self.w, y, z]
        ptC = [x, y+self.l, z]
        ptD = [x+self.w, y+self.l, z]
        ptE = [x, y, z+self.h]
        ptF = [x+self.w, y, z+self.h]
        ptG = [x, y+self.l, z+self.h]
        ptH = [x+self.w, y+self.l, z+self.h]
    
        pointsInit = [
            ptA,
            ptB,
            ptC,
            ptD,
            ptE,
            ptF,
            ptG,
            ptH
        ]
        ptA1 = [x-thresh, y-thresh, z-thresh]
        ptB1 = [x+self.w+thresh, y-thresh, z-thresh]
        ptC1 = [x-thresh, y+self.l+thresh, z-thresh]
        ptD1 = [x+self.w+thresh, y+self.l+thresh, z-thresh]
        ptE1 = [x-thresh, y-thresh, z+self.h+thresh]
        ptF1 = [x+self.w+thresh, y-thresh, z+self.h+thresh]
        ptG1 = [x-thresh, y+self.l+thresh, z+self.h+thresh]
        ptH1 = [x+self.w+thresh, y+self.l+thresh, z+self.h+thresh]
    
        pointsInit1 = [
            ptA1,
            ptB1,
            ptC1,
            ptD1,
            ptE1,
            ptF1,
            ptG1,
            ptH1
        ]
        lines = [
            [0, 1],
            [0, 2],
            [1, 3],
            [2, 3],
            [4, 5],
            [4, 6],
            [5, 7],
            [6, 7],
            [0, 4],
            [1, 5],
            [2, 6],
            [3, 7],
        ]
        lines1 = [
            [0, 1],
            [0, 2],
            [1, 3],
            [2, 3],
            [4, 5],
            [4, 6],
            [5, 7],
            [6, 7],
            [0, 4],
            [1, 5],
            [2, 6],
            [3, 7],
        ]
        
        colors = [[1, 0, 0] for i in range(len(lines))]
        colors1 = [[0, 1, 0] for i in range(len(lines1))]
        
        self.line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(pointsInit),
            lines=o3d.utility.Vector2iVector(lines),
        )
        self.line_set1 = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(pointsInit1),
            lines=o3d.utility.Vector2iVector(lines1),
        )
        self.line_set.colors = o3d.utility.Vector3dVector(colors)
        self.line_set1.colors = o3d.utility.Vector3dVector(colors1)
        rotation=self.line_set.get_rotation_matrix_from_xyz((math.radians(self.thetaX),math.radians(self.thetaY),math.radians(self.thetaZ)))
        #rotation1=self.line_set1.get_rotation_matrix_from_xyz((math.radians(self.thetaX),math.radians(self.thetaY),math.radians(self.thetaZ)))
        self.line_set.rotate(rotation, (self.centerx,self.centery,self.centerz))
        self.line_set1.rotate(rotation, (self.centerx,self.centery,self.centerz))
        
    def check_pts(self, points):
        pts_tfd = np.asarray(self.line_set.points)
        ptA, ptB, ptC, ptD, ptE, ptF, ptG, ptH = pts_tfd
        pts_tfd1 = np.asarray(self.line_set1.points)
        ptA1, ptB1, ptC1, ptD1, ptE1, ptF1, ptG1, ptH1 = pts_tfd1
        
        dir1 = (ptB-ptA)
        size1 = np.linalg.norm(dir1)
        dir1 = dir1 / size1
        
        dir2 = (ptC-ptA)
        size2 = np.linalg.norm(dir2)
        dir2 = dir2 / size2

        dir3 = (ptE-ptA)
        size3 = np.linalg.norm(dir3)
        dir3 = dir3 / size3
        
        dir11 = (ptB1-ptA1)
        size11 = np.linalg.norm(dir11)
        dir11 = dir11 / size11
        
        dir21 = (ptC1-ptA1)
        size21 = np.linalg.norm(dir21)
        dir21 = dir21 / size21

        dir31 = (ptE1-ptA1)
        size31 = np.linalg.norm(dir31)
        dir31 = dir31 / size31

        cube_center = np.array([self.centerx, self.centery, self.centerz]).reshape(1,3)
        self.pts_total = points.shape[0]
        pts_indices = np.arange(points.shape[0])
        indices = pts_indices.tolist()
        #print(points.shape, cube_center.shape)
        dir_vec =points - cube_center 
        res1 = np.where( (np.absolute(np.dot(dir_vec, dir1)) * 2) >= size1 )[0]
        res2 = np.where( (np.absolute(np.dot(dir_vec, dir2)) * 2) >= size2 )[0]
        res3 = np.where( (np.absolute(np.dot(dir_vec, dir3)) * 2) >= size3 )[0]
        
        res11 = np.where( (np.absolute(np.dot(dir_vec, dir11)) * 2) >= size11 )[0]
        res21 = np.where( (np.absolute(np.dot(dir_vec, dir21)) * 2) >= size21 )[0]
        res31 = np.where( (np.absolute(np.dot(dir_vec, dir31)) * 2) >= size31 )[0]

        fits =   list( set().union(res1, res2, res3) )
        fits1 = set(indices)- set(list( set().union(res11, res21, res31) ))
        self.inliers_total = len(np.intersect1d(fits,list(fits1)))
                
        return self.inliers_total
    

In [3]:
class GA:
    def __init__(self, crossover_rate):
        self.population=[]
        self.fitness =[]
        self.crossover_rate =crossover_rate
        self.params = ['x', 'y', 'z', 'w', 'l', 'h', 'thetaX', 'thetaY', 'thetaZ']
    def init_popultation(self, size):
        #self.population=[]
        self.size = size
        for _ in range(self.size):
            rand_x = random.uniform(-3, 3)
            rand_y = random.uniform(-3, 3)
            rand_z = random.uniform(-3, 3)
            rand_w = random.uniform(0.2, 4)
            rand_l = random.uniform(0.2, 4)
            rand_h = random.uniform(0.2, 4)
            rand_thetaX = random.uniform(0, 360)
            rand_thetaY = random.uniform(0, 360)
            rand_thetaZ = random.uniform(0, 360)
            self.population.append(cube(rand_x,rand_y,rand_z,rand_w,rand_l,rand_h,rand_thetaX,rand_thetaY,rand_thetaZ))
            
    def calc_fitness(self):
        self.fitness =[]
        for i in range(self.size):  
            self.fitness.append( 100*(self.population[i].inliers_total/(self.population[i].pts_total-self.population[i].inliers_total+1)))   
        #x = zip(self.fitness,self.population)
        #xs = sorted(x,reverse = True, key=lambda tup: tup[0])
        #self.fitness = [x[0] for x in xs]
        #self.population = [x[1] for x in xs]
        #print(self.fitness)
        return self.fitness
    
    def selection(self, fitness):
        
        total_fitness = sum(fitness)
        #print(total_fitness, 'fitness')
        if total_fitness !=0:
            #print(fitness, total_fitness)
            probabilities = np.asarray(fitness)/(total_fitness + sys.float_info.epsilon)
            #print('prob', probabilities)
            R = np.random.random(probabilities.shape)
            #print('rand', R)
            cumsum = np.cumsum(probabilities)
            cumsum = np.insert(cumsum, 0, 0)
            #print(cumsum)
            sorted_population =[None]*len(self.population)
            #print(sorted_population)
            for i in range(R.shape[0]):
                for j in range(0, cumsum.shape[0]-1):
                    #print(j, cumsum.shape,R.shape)
                    if R[i]<cumsum[j+1] and R[i]>=cumsum[j]:
                        sorted_population[i] = self.population[j]
                        break
            #print(sorted_population)
            R_crossover = np.random.random(probabilities.shape)
            parents_indices = []
            for i in range(len(sorted_population)):
                if R_crossover[i] < self.crossover_rate:
                    parents_indices.append(i)
        else:
            print('err')
            parents_indices = None
            sorted_population = None
        return parents_indices, sorted_population
    
    def crossover(self, parents_indices, sorted_population):
        R_exchange_pos = np.random.randint(0,8,[len(parents_indices)])
        if len(parents_indices) == 0:
            pass
            #print('no parents selected')
        elif len(parents_indices) == 1:
            pass
            #print('one parent selected')
        else:
            for j in range(len(parents_indices)):
                if j==(len(parents_indices)-1):
                    #print ('exchanging ',sorted_population[-1], 'with', sorted_population[0], 'at', R_exchange_pos[-1])
                    sorted_population[-1].__dict__[self.params[R_exchange_pos[-1]]] = sorted_population[0].__dict__[self.params[R_exchange_pos[-1]]]
                    #print('results in', sorted_population[-1])
                else:
                    #print ('exchanging ',sorted_population[parents_indices[j]], 'with', sorted_population[parents_indices[j+1]], 'at', R_exchange_pos[j])
                    sorted_population[parents_indices[j]].__dict__[self.params[R_exchange_pos[j]]] = sorted_population[parents_indices[j+1]].__dict__[self.params[R_exchange_pos[j]]]
                    #print('results in', sorted_population[parents_indices[j]])
        #for i in range(len(sorted_population)):
        #    print(sorted_population[i].__dict__)
        return sorted_population
    
    def mutation(self, population, mutation_rate):
        rng = default_rng()
        total_gen = len(population)*9
        mutation_chance = round(mutation_rate*total_gen)
        R_mutate_indices = rng.choice(len(population)*9, size=mutation_chance, replace=False)
        #print(R_mutate_indices)
        #R_mutate_indices = np.random.randint(0,population.size,[mutation_chance,1])
        #R_mutate_val = np.random.randint(0,30,[mutation_chance,1])
        
        for i in R_mutate_indices:
            #print(population[int(i/9)].__dict__[self.params[i%9]], 'before')
            population[int(i/9)].__dict__[self.params[i%9]] *= random.uniform(0,2)
            #print(population[int(i/9)].__dict__[self.params[i%9]], 'after')
            #print(i%9)
        #print(len(population))
        return population 
        #print(population)
        #return population
        

In [8]:
if __name__ == '__main__':
    #random.seed(2)
    vis = o3d.visualization.Visualizer()
    number_steps = 10
    best_fit = 0
    best_model = None
    algorithm = GA(0.1)
    algorithm.init_popultation(6)
    vis.create_window(width = 800, height = 600)
    #pcd = o3d.io.read_point_cloud("D:/Open3D/Open3D-master/examples/test_data/fragment.ply")
    pcd = o3d.io.read_point_cloud("models/cube_test1.ply")
    vis.add_geometry(pcd)
    geometry_list = list()
    
    
    #for i in range(len(algorithm.population)):
        
    for _ in range(number_steps):
        print(_, '\n')
        #random.shuffle(algorithm.population)
        #print(algorithm.population)
        vis.add_geometry(pcd)
        for i in range(len(algorithm.population)):
            vis.add_geometry(algorithm.population[i].line_set)
            vis.add_geometry(algorithm.population[i].line_set1)
            
            inliers = algorithm.population[i].check_pts(np.asarray(pcd.points))
        
        fit = algorithm.calc_fitness()
        if algorithm.fitness[0]>best_fit:
            best_fit = algorithm.fitness[0]
            print('Found better fintess: ',best_fit, 'at', _ )
            best_model = algorithm.population[0]
       
        parents_indices, sorted_population = algorithm.selection(fit)
        if parents_indices!=None and sorted_population!= None:
            sorted_popuation = algorithm.crossover(parents_indices, sorted_population)
            algorithm.population = algorithm.mutation(sorted_popuation, 0.1)
        else:
            algorithm.init_popultation(20)
        for i in range(len(algorithm.population)):
            print(algorithm.population[i].__dict__)
        vis.poll_events()
        vis.update_renderer()
        
        vis.clear_geometries()
        #for i in range(len(population)):
        #    print(population[i].__dict__)
    vis.add_geometry(pcd)
    vis.add_geometry(best_model.line_set)
    vis.add_geometry(best_model.line_set1)
    vis.run()
    vis.destroy_window()
    
   

0 

{'x': -1.5995174566163146, 'y': -4.925625435267549, 'z': -2.3707940588691474, 'w': 4.473622426262055, 'l': 1.9447530571564826, 'h': 1.4505208558495717, 'thetaX': 1.264552002033497, 'thetaY': 169.14336747802875, 'thetaZ': 63.324786216071466, 'centerx': 0.4750401149064749, 'centery': -1.6559822847356456, 'centerz': -1.4805937552261335, 'pts_total': 981, 'inliers_total': 34, 'line_set': LineSet with 12 lines., 'line_set1': LineSet with 12 lines.}
{'x': -1.5995174566163146, 'y': -4.925625435267549, 'z': -2.3707940588691474, 'w': 4.473622426262055, 'l': 1.9447530571564826, 'h': 1.4505208558495717, 'thetaX': 1.264552002033497, 'thetaY': 169.14336747802875, 'thetaZ': 63.324786216071466, 'centerx': 0.4750401149064749, 'centery': -1.6559822847356456, 'centerz': -1.4805937552261335, 'pts_total': 981, 'inliers_total': 34, 'line_set': LineSet with 12 lines., 'line_set1': LineSet with 12 lines.}
{'x': -1.5995174566163146, 'y': -4.925625435267549, 'z': -2.3707940588691474, 'w': 4.473622426262055

4 

{'x': -1.5666232490641558, 'y': -1.48879324520115, 'z': -3.139343109610755, 'w': 3.754149655064876, 'l': 2.0698623184241076, 'h': 2.0004356656391096, 'thetaX': 0.004722240289206361, 'thetaY': 246.79778385182837, 'thetaZ': 11.946559996340994, 'centerx': 0.4750401149064749, 'centery': -1.6559822847356456, 'centerz': -1.4805937552261335, 'pts_total': 981, 'inliers_total': 34, 'line_set': LineSet with 12 lines., 'line_set1': LineSet with 12 lines.}
{'x': -1.5666232490641558, 'y': -1.48879324520115, 'z': -3.139343109610755, 'w': 3.754149655064876, 'l': 2.0698623184241076, 'h': 2.0004356656391096, 'thetaX': 0.004722240289206361, 'thetaY': 246.79778385182837, 'thetaZ': 11.946559996340994, 'centerx': 0.4750401149064749, 'centery': -1.6559822847356456, 'centerz': -1.4805937552261335, 'pts_total': 981, 'inliers_total': 34, 'line_set': LineSet with 12 lines., 'line_set1': LineSet with 12 lines.}
{'x': -1.5666232490641558, 'y': -1.48879324520115, 'z': -3.139343109610755, 'w': 3.754149655064876

{'x': -0.03640671216253667, 'y': -0.5669400341478523, 'z': -3.5580271472321514, 'w': 1.62654672527706, 'l': 1.8222949472410483, 'h': 2.3661752859437377, 'thetaX': 0.002107179005308146, 'thetaY': 284.47152369973367, 'thetaZ': 12.035246063531217, 'centerx': 0.4750401149064749, 'centery': -1.6559822847356456, 'centerz': -1.4805937552261335, 'pts_total': 981, 'inliers_total': 34, 'line_set': LineSet with 12 lines., 'line_set1': LineSet with 12 lines.}
{'x': -0.03640671216253667, 'y': -0.5669400341478523, 'z': -3.5580271472321514, 'w': 1.62654672527706, 'l': 1.8222949472410483, 'h': 2.3661752859437377, 'thetaX': 0.002107179005308146, 'thetaY': 284.47152369973367, 'thetaZ': 12.035246063531217, 'centerx': 0.4750401149064749, 'centery': -1.6559822847356456, 'centerz': -1.4805937552261335, 'pts_total': 981, 'inliers_total': 34, 'line_set': LineSet with 12 lines., 'line_set1': LineSet with 12 lines.}
{'x': -0.03640671216253667, 'y': -0.5669400341478523, 'z': -3.5580271472321514, 'w': 1.626546725

In [ ]:
vis.create_window(width = 800, height = 600)
vis.add_geometry(pcd)
vis.add_geometry(best_model.line_set)
vis.add_geometry(best_model.line_set1)
vis.run()
vis.destroy_window()

In [132]:
cubik.__dict__['y'] = 1212

In [134]:
cubik.__dict__

{'x': 0,
 'y': 1212,
 'z': 2,
 'w': 3,
 'l': 4,
 'h': 5,
 'thetaX': 6,
 'thetaY': 7,
 'thetaZ': 8,
 'centerx': 1.5,
 'centery': 3.0,
 'centerz': 4.5,
 'pts_total': 0,
 'inliers_total': 0,
 'line_set': LineSet with 12 lines.,
 'line_set1': LineSet with 12 lines.}